In [1]:
import pandas as pd
import numpy as np

import math
import statistics as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind

## Part 1

In [2]:
data = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
data

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [3]:
# Standardisation of column names
cols = [col.lower().replace(" ", "_") for col in data.columns]
data.columns = cols

In [4]:
# Getting a first idea about the numerical column of the set
data.describe()

,etching_rate
count,15.000000
mean,6.782667
std,1.228643
min,5.430000
25%,5.845000
50%,6.240000
75%,7.725000
max,9.200000


In [5]:
# Taking a look at the separate categories in the categorical column
data.power_.unique()

array(['160 W', '180 W', '200 W'], dtype=object)

In [6]:
# Aggregating based on the categories of power variable and calculating the mean for each one
means_df = pd.DataFrame(data.groupby('power_').agg(np.mean))
means_df

,etching_rate
power_,
160 W,5.792
180 W,6.238
200 W,8.318


- H<sub>0</sub>: mean_160 = mean_180 = mean_200, where mean_160, mean_180 and mean_200 are the means for the etching rates in regards to their respective wattage.
- H<sub>1</sub>: any of the means is significantly different than any of the rest means.
- Significance level: α = 0.05
- The degrees of freedom of the model are df_model = 2 because the  is equal to the number of columns in the dataset.
- The error terms are provided by the model and are:
<br> --> the residual for the sum of squares and 
<br> --> the redidual for the mean of squares
- The degrees of freedom for the set are as follows:

In [7]:
# with df2 being the total DoF
df1 = data.power_.nunique() - 1
df2 = len(data) - data.power_.nunique()
print("df1 =", df1)
print("df2 =", df2)

df1 = 2
df2 = 12


## Part 2

In [8]:
# Setting up and running ANOVA
model = ols('etching_rate ~ C(power_)',data=data).fit()    # C(Display_design): defines the variable as categorical, in order to avoid confusion in calculations
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(power_),2.0,18.176653,9.088327,36.878955,0.000008
Residual,12.0,2.957240,0.246437,NaN,NaN


According to the model's results, pvalue = 8 * 10<sup>-6</sup>, which means that pvalue < α and thus there is significant differences at least between the measurements of the etching rate of 2 from the 3 settings of wattage. ANOVA does not provide any extra insight than that, but looking at the average etching rates, the 200 W category offers an apparent and much more distinct increase in the etching rate of the lasers. The 2 first categories can be tested against the latter to see if this observation is indeed a cause for the rejection of the null hypothesis.

<br>But before this kind of analysis is run, a better look into some metrics for the 3 wattage settings might help decide if this assumption is worth looking into:

In [9]:
data.pivot(columns='power_').describe().loc[['count', 'min', 'mean', 'std', 'max']]   # .loc[] can be used to select only the stats needed because .describe() creates df with stats names as index values

etching_rate                    
power_        160 W     180 W     200 W
count      5.000000  5.000000  5.000000
min        5.430000  5.660000  7.550000
mean       5.792000  6.238000  8.318000
std        0.319875  0.434304  0.669604
max        6.220000  6.710000  9.200000

In [10]:
# Difference between the means of 200 and 180 W measurements
print("Difference in", means_df.loc['200 W'] - means_df.loc['180 W'])

Difference in etching_rate    2.08
dtype: float64


In [11]:
# Difference between the means of 180 and 160 W measurements
print("Difference in", means_df.loc['180 W'] - means_df.loc['160 W'])

Difference in etching_rate    0.446
dtype: float64


Clearly, the standard deviation of the 200 W category is not great enough for there to be a concern of it spilling into the 180 W category. Additionally, it seems to have substantially higher min, mean and max values. Thus, it is worth it to test the assumption above.
<br>The samples are independent since each measurement can only belong to one category, their size is less than 30 and the variance is unknown.
<br>Two-sample ttests will be applied for the 2 potential pairs of samples.
<br>H<sub>0</sub>: μ = 8.32
<br>H<sub>1</sub>: μ != 8.32
<br>α = 0.05
<br>Since the alternate hypothesis does not specify the kind of difference, it refers to a 2-tailed test.

In [12]:
# Creating measurement lists for the 3 categories
list_a = data[data['power_'] == "200 W"]['etching_rate']

In [13]:
# using the clever for loop used by Xisca with a slight modification
for power in data.power_.unique():
    if power != "200 W":
        list_b = data[data['power_'] == power]['etching_rate']
        print(ttest_ind(list_a, list_b))

Ttest_indResult(statistic=7.611403634613074, pvalue=6.237977344615716e-05)
Ttest_indResult(statistic=5.827496614588661, pvalue=0.0003926796476049085)


So according to the ttest for the pair 200 W & 160 W, the pvalue is 6.237977344615716e-05 < α and for the second pair pvalue = 0.00039 < 0.05. Consequently, both of the those wattages bear significant differences to the 200 W setting in regards to the etching rate of the lasers.